In [1]:
from relationalai.semantics import Model, std, Integer, String, Float
from datetime import datetime
import relationalai.semantics as rai

model = Model("arq_starter_queries")


### 1. Temporality

In [2]:
Observation = model.Concept("Observation", identify_by={"id": Integer})

EventDateTime = model.Concept("EventDateTime", extends=[datetime])


# Define properties
Observation.event_datetime = model.Property("{Observation} occurred on {EventDateTime}")

test_timestamp = datetime.strptime("2025-11-21T10:00:00z", "%Y-%m-%dT%H:%M:%Sz")

rai.define(
    obs1 := Observation.new(id=1),
    obs1.event_datetime(EventDateTime(test_timestamp))
)


rai.select(Observation, Observation.id, Observation.event_datetime).inspect()

              observation id       eventdatetime
0  rZ7RcIobOaQAZ955Xc3Yxw  1 2025-11-21 10:00:00


In [3]:
Observation.day_of_year = model.Property("{Observation} occurred on day {day_of_year:Integer} of the year")
Observation.year = model.Property("{Observation} occurred in year {year:Integer}")


rai.define(
    Observation.year(
        std.datetime.datetime.year(Observation.event_datetime)
        )
)

rai.define(
    Observation.day_of_year(
        std.datetime.datetime.dayofyear(Observation.event_datetime)
        )
)

rai.select(Observation, Observation.id, Observation.event_datetime, Observation.year, Observation.day_of_year).inspect()


              observation id       eventdatetime  year day_of_year
0  rZ7RcIobOaQAZ955Xc3Yxw  1 2025-11-21 10:00:00  2025         325


Another approach is to define these properties for `EventDateTime` entities. For example:

In [4]:
EventDateTime.year = model.Property("{EventDateTime} is in year {year:Integer}")

rai.define(
    EventDateTime.year(
        std.datetime.datetime.year(EventDateTime)
        )
)

rai.select(Observation, Observation.event_datetime.year).inspect()

              observation  year
0  rZ7RcIobOaQAZ955Xc3Yxw  2025


We can also define properties that are temporally indexed. For example:

In [5]:
Taxon = model.Concept("Taxon", identify_by={"id": Integer})
Taxon.scientific_name_at_time = model.Property("{Taxon} had scientific name {String} at {EventDateTime}")

Using multiple properties we can capture intervals as well.

In [6]:
TaxonNameValidity = model.Concept("TaxonNameValidity")

TaxonNameValidity.taxon = model.Property("{TaxonNameValidity} refers to {Taxon}")
TaxonNameValidity.name = model.Property("{TaxonNameValidity} has name {String}")
TaxonNameValidity.valid_from = model.Property("{TaxonNameValidity} valid from {EventDateTime}")
TaxonNameValidity.valid_to = model.Property("{TaxonNameValidity} valid until {EventDateTime}")

TaxonNameValidity.identify_by(TaxonNameValidity.taxon, TaxonNameValidity.name, TaxonNameValidity.valid_from)

# Query the validity periods of a given taxon name
rai.select(
    TaxonNameValidity.valid_from,
    TaxonNameValidity.valid_to
).where(
    TaxonNameValidity.name == "Vaccinium angulatum J.J.Sm."
)

Query all active taxon at a given datetime


In [ ]:
rai.select(
    TaxonNameValidity.taxon,
    TaxonNameValidity.name
).where(
    TaxonNameValidity.valid_from <= EventDateTime(test_timestamp),
    TaxonNameValidity.valid_to > EventDateTime(test_timestamp)
)

### 2. Provenance


In [8]:
# Define provenance concepts
Dataset = model.Concept("Dataset", identify_by={"dataset_id": rai.String})


# Add provenance to observations
Observation.source_dataset = model.Property("{Observation} sourced from {Dataset}")

rai.define(
    Observation.source_dataset(Dataset.new(dataset_id="inaturalist"))
).where(
    Observation.filter_by(id=1)
)

rai.select(Observation.id, Observation.source_dataset.dataset_id).inspect()

  id  dataset_id
0  1         NaN


In [ ]:
Observation.has_trust_level = model.Property("{Observation} has trust level {trust_level:Float}")

Observation.trust_level = model.Property("{TrustLeve} belongs to {Observation} ")

rai.define(Observation.trust_level(0.9)).where(
    Observation.source_dataset.dataset_id == "inaturalist"
)

  id  trust_level
0  1          NaN


https://www.orm.net/pdf/AutomatedVerbalization.pdf


https://www.brcommunity.com/articles.php?id=b138

In [ ]:
ScientificName = model.Concept("ScientificName", extends=[rai.String])

rai.select(
    Taxon.id
).where(
    Taxon.scientific_name == ScientificName("Vaccinium angulatum J.J.Sm.")
)

In [ ]:
Taxon.scientic_name_string = model.Property("{Taxon} has scientific name string {String}")
rai.select(
    Taxon.id
).where(
    Taxon.scientific_name_string == "Vaccinium angulatum J.J.Sm."
)

### 9. What sorts of compound facts are possible?

In [ ]:
Observation.phenological_state = model.Property("{Observation} observed {Species} with {phenophase:String} at {coverage_percent:Integer}%")
